In [ ]:
# | default_exp _cli.pred

In [ ]:
from airt._testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import os

import pandas as pd
import typer
from tabulate import tabulate
from typer import echo

from airt._cli import helper
from airt._constant import CLIENT_DB_PASSWORD, CLIENT_DB_USERNAME
from airt._logger import get_logger, set_level

In [ ]:
import logging
import tempfile
import time
from contextlib import contextmanager
from datetime import timedelta
from pathlib import Path

import boto3
import pytest
from azure.identity import DefaultAzureCredential
from azure.mgmt.storage import StorageManagementClient
from typer.testing import CliRunner

import airt._sanitizer
from airt._constant import SERVICE_PASSWORD, SERVICE_TOKEN, SERVICE_USERNAME
from airt.client import Client, DataBlob, User

In [ ]:
# | exporti

app = typer.Typer(
    help="A set of commands for managing and downloading the predictions."
)

In [ ]:
runner = CliRunner()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
set_level(logging.WARNING)

In [ ]:
# Testing logger settings

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
TEST_S3_URI = "s3://test-airt-service/ecommerce_behavior_notebooks"
TEST_AZURE_PUSH_URI = (
    "https://testairtservice.blob.core.windows.net/test-client-push-container"
)
RANDOM_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def ls(
    offset: int = typer.Option(
        0,
        "--offset",
        "-o",
        help="The number of predictions to offset at the beginning. If None, then the default value **0** will be used.",
    ),
    limit: int = typer.Option(
        100,
        "--limit",
        "-l",
        help="The maximum number of predictions to return from the server. If None, "
        "then the default value **100** will be used.",
    ),
    disabled: bool = typer.Option(
        False,
        "--disabled",
        help="If set to **True**, then only the deleted predictions will be returned. Else, the default value "
        "**False** will be used to return only the list of active predictions.",
    ),
    completed: bool = typer.Option(
        False,
        "--completed",
        help="If set to **True**, then only the predictions that are successfully downloaded "
        "to the server will be returned. Else, the default value **False** will be used to "
        "return all the predictions.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only prediction uuids separated by space.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the list of predictions."""

    from airt.client import Prediction

    predx = Prediction.ls(
        offset=offset, limit=limit, disabled=disabled, completed=completed
    )

    df = Prediction.as_df(predx)

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df, "quite_column_name": "prediction_uuid"}

In [ ]:
def assert_has_help(xs: List[str]):
    result = runner.invoke(app, xs + ["--help"])

    display(result.stdout)
    assert " ".join(xs) in result.stdout

In [ ]:
assert_has_help(["ls"])

'Usage: ls [OPTIONS]\n\n  Return the list of predictions.\n\nOptions:\n  -o, --offset INTEGER            The number of predictions to offset at the\n                                  beginning. If None, then the default value\n                                  **0** will be used.  [default: 0]\n  -l, --limit INTEGER             The maximum number of predictions to return\n                                  from the server. If None, then the default\n                                  value **100** will be used.  [default: 100]\n  --disabled                      If set to **True**, then only the deleted\n                                  predictions will be returned. Else, the\n                                  default value **False** will be used to return\n                                  only the list of active predictions.\n  --completed                     If set to **True**, then only the predictions\n                                  that are successfully downloaded to the server\

In [ ]:
# Helper context manager for testing

_airt_service_token = None


@contextmanager
def set_airt_service_token_envvar():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
_prediction = None


@contextmanager
def generate_prediction(force_create: bool = False):
    global _prediction
    if _prediction is None or force_create:
        db = DataBlob.from_s3(
            uri=TEST_S3_URI,
            access_key=os.environ["AWS_ACCESS_KEY_ID"],
            secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
            cloud_provider="aws",
            region="eu-west-1",
        )

        db.progress_bar()

        ds = db.to_datasource(
            file_type="parquet", index_column="user_id", sort_by="event_time"
        )

        display(f"{ds.uuid=}")

        ds.progress_bar()

        model = ds.train(
            client_column="user_id",
            target_column="category_code",
            target="*checkout",
            predict_after=timedelta(hours=3),
        )

        model.wait()

        _prediction = model.predict()
        _prediction.progress_bar()

    yield _prediction

In [ ]:
# Tests for prediction ls
# Testing positive scenario. Saving the token in env variable


def get_uuids_from_result(result) -> List[int]:
    return [uuid for uuid in result.stdout[:-1].split("\n")]


with set_airt_service_token_envvar():
    with generate_prediction(force_create=True) as pred:
        # Without quiet
        format_str = "{'ready': '{}', 'prediction_uuid': '{}'}"
        result = runner.invoke(app, ["--format", format_str])
        display(result.stdout)

        assert "ready" in result.stdout
        assert result.exit_code == 0

        # With quiet
        result = runner.invoke(app, ["-q"])
        display(result.stdout)

        assert result.exit_code == 0
        uuids = get_uuids_from_result(result)
        display(f"{uuids=}")

'_airt_service_token is None, getting a token...'

100%|██████████| 1/1 [00:10<00:00, 10.12s/it]


"ds.uuid='fe0cd7a8-e1c6-4166-bbb9-4a7c0fd3d6e5'"

100%|██████████| 3/3 [00:05<00:00,  1.70s/it]


'ready    prediction_uuid\nTrue     6b859549-dad2-449f-ae41-0eda1dae0d9a\nTrue     8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1\n'

'6b859549-dad2-449f-ae41-0eda1dae0d9a\n8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1\n'

"uuids=['6b859549-dad2-449f-ae41-0eda1dae0d9a', '8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1']"

In [ ]:
# Tests for prediction ls
# Testing positive scenario.
# Testing by passing different values for  limit


with set_airt_service_token_envvar():
    for limit in [1, 10, 1000]:
        offset = 1
        result = runner.invoke(app, ["--offset", offset, "--limit", limit, "-q"])

        assert result.exit_code == 0

        ids = get_uuids_from_result(result)
        display(f"{ids=}")
        assert limit >= len(ids) >= 0

"ids=['8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1']"

"ids=['8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1']"

"ids=['8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1']"

In [ ]:
# Tests for prediction ls
# Testing positive scenario.
# Testing by passing large value for offset.

with set_airt_service_token_envvar():
    limit = 10
    offset = 1_000_000
    result = runner.invoke(app, ["--offset", offset, "--limit", limit])

    assert result.exit_code == 0

    display(result.stdout)

'prediction_uuid    created    ready\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def details(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the details of a prediction."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)
    df = pred.details()

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
assert_has_help(["details"])

'Usage: root details [OPTIONS] UUID\n\n  Return the details of a prediction.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for details
# Testing positive scenario

# Helper function to extract ID


def extract_id(res) -> str:
    r = (res.split("\n")[1]).strip()
    return r.split(" ")[0]


with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        # Getting Details of the prediction
        pred_uuid = pred.uuid
        format_str = "{'prediction_uuid': '{}'}"
        result = runner.invoke(app, ["details", pred_uuid, "--format", format_str])

        display(result.stdout)

        assert result.exit_code == 0
        assert result.stdout == f"{pred_uuid}\n", f"{result_id=} {pred_uuid=}"

'8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1\n'

In [ ]:
# Tests for details
# Testing negative scenario. Passing invalie pred_uuid

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        result = runner.invoke(app, ["details", RANDOM_UUID_FOR_TESTING])

        display(result.stdout)

        assert result.exit_code == 1

'Error: The prediction uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def rm(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output the deleted Prediction uuid only.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Delete a prediction from the server."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)
    df = pred.delete()

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df, "quite_column_name": "prediction_uuid"}

In [ ]:
assert_has_help(["rm"])

'Usage: root rm [OPTIONS] UUID\n\n  Delete a prediction from the server.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  -q, --quiet        Output the deleted Prediction uuid only.\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for prediction rm
# Testing positive scenario with quite

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        format_str = "{'prediction_uuid': '{}'}"
        pred_uuid = pred.uuid
        result = runner.invoke(app, ["rm", pred_uuid, "-f", format_str])
        deleted_id = result.stdout[:-1]

        display(deleted_id)

        assert result.exit_code == 0
        assert deleted_id == pred_uuid

        # List the existing Prediction uuids in server and make sure the deleted id is not present
        ls_result = runner.invoke(app, ["ls", "-q"])
        ls_ids = get_uuids_from_result(ls_result)

        display(ls_ids)
        assert deleted_id not in ls_ids

        # Testing negative scenario. Deleting already deleted prediction
        result = runner.invoke(app, ["rm", deleted_id, "-q"])

        # Testing negative scenario. Getting the details of the deleted prediction
        result = runner.invoke(app, ["details", deleted_id])

'8dac1fee-dbe6-42e4-aab3-bf921a9bb6c1'

['6b859549-dad2-449f-ae41-0eda1dae0d9a']

In [ ]:
# Tests for prediction rm
# Testing negative scenario. Deleting invalid prediction

with set_airt_service_token_envvar():
    result = runner.invoke(app, ["rm", RANDOM_UUID_FOR_TESTING])

    display(result.stdout)

'Error: The prediction uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command("to-pandas")
@helper.requires_auth_token
def to_pandas(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Return the prediction results as a pandas DataFrame."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)

    df = pred.to_pandas()

    typer.echo(tabulate(df, headers="keys", tablefmt="plain", showindex=False))  # type: ignore

In [ ]:
assert_has_help(["to-pandas"])

'Usage: root to-pandas [OPTIONS] UUID\n\n  Return the prediction results as a pandas DataFrame.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  -d, --debug  Set logger level to DEBUG and output everything.\n  --help       Show this message and exit.\n'

In [ ]:
# Tests for prediction to-pandas
# Testing positive scenario

with set_airt_service_token_envvar():
    with generate_prediction(force_create=True) as pred:
        result = runner.invoke(app, ["to-pandas", pred.uuid])

        display(result.stdout)

        assert result.exit_code == 0
        assert "Score" in result.stdout

100%|██████████| 1/1 [00:10<00:00, 10.11s/it]


"ds.uuid='a81aeac7-7984-493a-acc3-9d6949afa40a'"

100%|██████████| 3/3 [00:05<00:00,  1.69s/it]


'   Score\n0.979853\n0.979157\n0.979055\n0.978915\n0.97796\n0.004043\n0.00389\n0.001346\n0.001341\n0.001139\n'

In [ ]:
# Tests for prediction to-pandas
# Testing negative scenario

with set_airt_service_token_envvar():
    result = runner.invoke(app, ["to-pandas", RANDOM_UUID_FOR_TESTING])

    display(result.stdout)

'Error: The prediction uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command("to-s3")
@helper.requires_auth_token
def to_s3(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    uri: str = typer.Option(..., help="The target S3 bucket uri."),
    access_key: Optional[str] = typer.Option(
        None,
        help="Access key for the target S3 bucket. If **None** (default value), then the value from **AWS_ACCESS_KEY_ID** environment variable is used.",
    ),
    secret_key: Optional[str] = typer.Option(
        None,
        help="Secret key for the target S3 bucket. If **None** (default value), then the value from **AWS_SECRET_ACCESS_KEY** environment variable is used.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output status only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Push the prediction results to the target AWS S3 bucket."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)

    status = pred.to_s3(uri=uri, access_key=access_key, secret_key=secret_key)

    if quiet:
        status.wait()
        typer.echo(f"{pred.uuid}")
    else:
        typer.echo(
            f"Pushing the results for Prediction uuid: {pred.uuid} to the s3 bucket."
        )
        status.progress_bar()

In [ ]:
assert_has_help(["to-s3"])

'Usage: root to-s3 [OPTIONS] UUID\n\n  Push the prediction results to the target AWS S3 bucket.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  --uri TEXT         The target S3 bucket uri.  [required]\n  --access-key TEXT  Access key for the target S3 bucket. If **None** (default\n                     value), then the value from **AWS_ACCESS_KEY_ID**\n                     environment variable is used.\n  --secret-key TEXT  Secret key for the target S3 bucket. If **None** (default\n                     value), then the value from **AWS_SECRET_ACCESS_KEY**\n                     environment variable is used.\n  -q, --quiet        Output status only.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for prediction to-s3
# Testing positive scenario

user_details = User.details()
DEV_BUCKET_NAME = f'{os.environ["STORAGE_BUCKET_PREFIX"]}-eu-west-1'
TEST_OBJECT_NAME = f"{user_details['uuid']}/test_CLI_prediction_to_s3"
PREDICTION_TO_S3_URL = f"s3://{DEV_BUCKET_NAME}/{TEST_OBJECT_NAME}"

# Create a new key in the s3 bucket
s3_client = boto3.client("s3")

try:
    s3_client.create_bucket(
        Bucket=DEV_BUCKET_NAME,
        CreateBucketConfiguration={"LocationConstraint": "eu-west-1"},
    )
except s3_client.exceptions.BucketAlreadyOwnedByYou as e:
    logger.info("Bucket already created")

s3_client.put_object(Bucket=DEV_BUCKET_NAME, Key=(TEST_OBJECT_NAME + "/"))

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        pred_uuid = pred.uuid

        # without -q
        result = runner.invoke(app, ["to-s3", pred_uuid, "--uri", PREDICTION_TO_S3_URL])

        display(result.stdout)
        assert result.exit_code == 0
        assert f"Pushing the results for Prediction uuid: {pred_uuid}" in result.stdout

        # with -q
        result = runner.invoke(
            app, ["to-s3", pred_uuid, "--uri", PREDICTION_TO_S3_URL, "-q"]
        )

        display(result.stdout)
        assert result.exit_code == 0
        assert f"{pred_uuid}" in result.stdout


# Check in s3 if the uploaded files are present
time.sleep(10)
response = s3_client.list_objects(Bucket=DEV_BUCKET_NAME, Prefix=TEST_OBJECT_NAME)
actual_s3_contents = [content.get("Key") for content in response.get("Contents", [])]
expected_s3_contents = [
    f"{TEST_OBJECT_NAME}/",
    f"{TEST_OBJECT_NAME}/part.0.parquet",
]

assert len(actual_s3_contents) == 2, len(actual_s3_contents)
assert actual_s3_contents == expected_s3_contents, actual_s3_contents
display(f"{actual_s3_contents=}")

# Finally, delete the object in s3
for k in actual_s3_contents:
    s3_client.delete_object(Bucket=DEV_BUCKET_NAME, Key=k)

response = s3_client.list_objects(Bucket=DEV_BUCKET_NAME, Prefix=TEST_OBJECT_NAME)
s3_contents = [content.get("Key") for content in response.get("Contents", [])]

assert s3_contents == [], s3_contents
display(f"{s3_contents=}")

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the s3 bucket.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|██████████| 1/1 [00:05<00:00,  5.08s/it]\r100%|██████████| 1/1 [00:05<00:00,  5.09s/it]\n'

'863a392f-849b-4da7-b86e-4c0ce85eb221\n'

"actual_s3_contents=['bbe0e09b-9c56-434c-a101-c47adae2a0c8/test_CLI_prediction_to_s3/', 'bbe0e09b-9c56-434c-a101-c47adae2a0c8/test_CLI_prediction_to_s3/part.0.parquet']"

's3_contents=[]'

In [ ]:
# Tests for prediction to-s3
# Testing negative scenario

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        uri = "s3://random-bucket-name/random-object-name"
        access_key = "fake_access_key"
        secret_key = "fake_secret_key"

        result = runner.invoke(
            app,
            [
                "to-s3",
                pred.uuid,
                "--uri",
                uri,
                "--access-key",
                access_key,
                "--secret-key",
                secret_key,
            ],
        )

        display(result.stdout)
        assert result.exit_code == 1
        assert (
            f"The AWS Access Key Id you provided does not exist in our records"
            in result.stdout
        )

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the s3 bucket.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\nError: An error occurred (InvalidAccessKeyId) when calling the ListObjects operation: The AWS Access Key Id you provided does not exist in our records.\n'

In [ ]:
# | exporti


@app.command("to-azure-blob-storage")
@helper.requires_auth_token
def to_azure_blob_storage(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    uri: str = typer.Option(..., help="Target Azure Blob Storage uri."),
    credential: str = typer.Option(
        ...,
        "--credential",
        "-c",
        help="Credential to access the Azure Blob Storage.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output status only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Push the prediction results to the target Azure Blob Storage."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)
    status = pred.to_azure_blob_storage(uri=uri, credential=credential)

    if quiet:
        status.wait()
        typer.echo(f"{pred.uuid}")
    else:
        typer.echo(
            f"Pushing the results for Prediction uuid: {pred.uuid} to the Azure Blob Storage."
        )
        status.progress_bar()

In [ ]:
assert_has_help(["to-azure-blob-storage"])

'Usage: root to-azure-blob-storage [OPTIONS] UUID\n\n  Push the prediction results to the target Azure Blob Storage.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  --uri TEXT             Target Azure Blob Storage uri.  [required]\n  -c, --credential TEXT  Credential to access the Azure Blob Storage.\n                         [required]\n  -q, --quiet            Output status only.\n  -d, --debug            Set logger level to DEBUG and output everything.\n  --help                 Show this message and exit.\n'

In [ ]:
# Tests for to-azure-blob-storage
# Positive Scenario: Passing credential in arguments

storage_client = StorageManagementClient(
    DefaultAzureCredential(), os.environ["AZURE_SUBSCRIPTION_ID"]
)
keys = storage_client.storage_accounts.list_keys("test-airt-service", "testairtservice")
credential = keys.keys[0].value

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        # without -q
        result = runner.invoke(
            app,
            [
                "to-azure-blob-storage",
                pred.uuid,
                "--uri",
                TEST_AZURE_PUSH_URI,
                "--credential",
                credential,
            ],
        )
        display(result.stdout)
        assert result.exit_code == 0
        assert f"Pushing the results for Prediction uuid: {pred.uuid}" in result.stdout

        # with -q
        result = runner.invoke(
            app,
            [
                "to-azure-blob-storage",
                pred.uuid,
                "--uri",
                TEST_AZURE_PUSH_URI,
                "--credential",
                credential,
                "-q",
            ],
        )

        display(result.stdout)
        assert result.exit_code == 0
        assert f"{pred.uuid}" in result.stdout

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the Azure Blob Storage.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|██████████| 1/1 [00:05<00:00,  5.09s/it]\r100%|██████████| 1/1 [00:05<00:00,  5.09s/it]\n'

'863a392f-849b-4da7-b86e-4c0ce85eb221\n'

In [ ]:
# Tests for to-azure-blob-storage
# Negative Scenario: Passing invalid uri


with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        invalid_uri = "https://invalid-push-blob-storage-path"
        result = runner.invoke(
            app,
            [
                "to-azure-blob-storage",
                pred.uuid,
                "--uri",
                invalid_uri,
                "--credential",
                credential,
            ],
        )
        display(result.stdout)
        assert result.exit_code == 1

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the Azure Blob Storage.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\nError: Unable to determine account name for shared key credential.\n'

In [ ]:
# | exporti


@app.command("to-local")
@helper.requires_auth_token
def to_local(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    path: str = typer.Option(..., help="Local directory path."),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output status only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Download the prediction results to a local directory."""

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)

    if quiet:
        pred.to_local(path=path, show_progress=False)
        typer.echo(f"{pred.uuid}")
    else:
        typer.echo(f"Downloading prediction results for uuid: {pred.uuid}.")
        pred.to_local(path=path)

In [ ]:
assert_has_help(["to-local"])

'Usage: root to-local [OPTIONS] UUID\n\n  Download the prediction results to a local directory.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  --path TEXT  Local directory path.  [required]\n  -q, --quiet  Output status only.\n  -d, --debug  Set logger level to DEBUG and output everything.\n  --help       Show this message and exit.\n'

In [ ]:
# Tests for prediction to-local
# Testing positive scenario

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        with tempfile.TemporaryDirectory(prefix="test_to_local_") as d:
            assert os.listdir(d) == []
            display(list(os.listdir(d)))

            result = runner.invoke(app, ["to-local", pred.uuid, "--path", d])

            display(result.stdout)
            assert result.exit_code == 0
            assert (
                f"Downloading prediction results for uuid: {pred.uuid}."
                in result.stdout
            )

            downloaded_files = sorted(list(os.listdir(d)))
            assert downloaded_files == ["part.0.parquet"], downloaded_files
            display(f"{downloaded_files=}")

        # with -q flag
        with tempfile.TemporaryDirectory(prefix="test_to_local_") as d:
            assert os.listdir(d) == []
            display(list(os.listdir(d)))

            result = runner.invoke(app, ["to-local", pred.uuid, "--path", d, "-q"])

            display(result.stdout)
            assert result.exit_code == 0
            assert f"{pred.uuid}" in result.stdout

            downloaded_files = sorted(list(os.listdir(d)))
            assert downloaded_files == ["part.0.parquet"], downloaded_files
            display(f"{downloaded_files=}")

[]

'Downloading prediction results for uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|██████████| 1/1 [00:00<00:00,  1.25it/s]\r100%|██████████| 1/1 [00:00<00:00,  1.25it/s]\n'

"downloaded_files=['part.0.parquet']"

[]

'863a392f-849b-4da7-b86e-4c0ce85eb221\n'

"downloaded_files=['part.0.parquet']"

In [ ]:
# Tests for prediction to-local
# Testing negative scenario

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        d = Path("my-fake-path")

        result = runner.invoke(app, ["to-local", pred.uuid, "--path", d])

        display(result.stdout)
        assert result.exit_code == 1
        assert "my-fake-path" in result.stdout

"Downloading prediction results for uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221.\n\r  0%|          | 0/1 [00:00<?, ?it/s]Error: [Errno 2] No such file or directory: 'my-fake-path/part.0.parquet'\n"

In [ ]:
# | exporti


@app.command("to-mysql")
@helper.requires_auth_token
def to_mysql(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    host: str = typer.Option(..., help="Database host name."),
    database: str = typer.Option(..., help="Database name."),
    table: str = typer.Option(..., help="Table name."),
    port: int = typer.Option(
        3306,
        help="Host port number. If not passed, then the default value **3306** will be used.",
    ),
    username: Optional[str] = typer.Option(
        None,
        "--username",
        "-u",
        help="Database username. If not passed, then the value set in the environment variable"
        f" **{CLIENT_DB_USERNAME}** will be used else the default value **root** will be used.",
    ),
    password: Optional[str] = typer.Option(
        None,
        "--password",
        "-p",
        help="Database password. If not passed, then the value set in the environment variable"
        f' **{CLIENT_DB_PASSWORD}** will be used else the default value "" will be used.',
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output status only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Push the prediction results to a mysql database.

    If the database requires authentication, pass the username/password as commandline arguments or store it in
    the **AIRT_CLIENT_DB_USERNAME** and **AIRT_CLIENT_DB_PASSWORD** environment variables.
    """

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)

    status = pred.to_mysql(
        host=host,
        database=database,
        table=table,
        port=port,
        username=username,
        password=password,
    )

    if quiet:
        status.wait()
        typer.echo(f"{pred.uuid}")
    else:
        typer.echo(
            f"Pushing the results for Prediction uuid: {pred.uuid} to the mysql database."
        )
        status.progress_bar()

In [ ]:
assert_has_help(["to-mysql"])

'Usage: root to-mysql [OPTIONS] UUID\n\n  Push the prediction results to a mysql database.\n\n  If the database requires authentication, pass the username/password as\n  commandline arguments or store it in the **AIRT_CLIENT_DB_USERNAME** and\n  **AIRT_CLIENT_DB_PASSWORD** environment variables.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  --host TEXT          Database host name.  [required]\n  --database TEXT      Database name.  [required]\n  --table TEXT         Table name.  [required]\n  --port INTEGER       Host port number. If not passed, then the default value\n                       **3306** will be used.  [default: 3306]\n  -u, --username TEXT  Database username. If not passed, then the value set in\n                       the environment variable **AIRT_CLIENT_DB_USERNAME** will\n                       be used else the default value **root** will be used.\n  -p, --password TEXT  Database password. If not passed, then the value set in\n                   

In [ ]:
# Tests for prediction to-mysql
# Testing positive scenario

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        cmd = [
            "to-mysql",
            pred.uuid,
            "--host",
            os.environ["DB_HOST"],
            "--database",
            os.environ["DB_DATABASE"],
            "--table",
            "prediction_to_mysql",
            "-u",
            os.environ["DB_USERNAME"],
            "-p",
            os.environ["DB_PASSWORD"],
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0

        assert (
            f"Pushing the results for Prediction uuid: {pred.uuid} to the mysql database"
            in str(result.stdout)
        )

        cmd = [
            "to-mysql",
            pred.uuid,
            "--host",
            os.environ["DB_HOST"],
            "--database",
            os.environ["DB_DATABASE"],
            "--table",
            "prediction_to_mysql",
            "-u",
            os.environ["DB_USERNAME"],
            "-p",
            os.environ["DB_PASSWORD"],
            "-q",
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0

        assert f"{pred.uuid}" in str(result.stdout)

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the mysql database.\n\n\r  0%|          | 0/1 [00:00<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:05<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:10<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:15<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:20<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:25<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:30<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:35<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:40<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:45<?, ?it/s]\x1b[A\n\r  0%|          | 0/1 [00:50<?, ?it/s]\x1b[A\n\r100%|██████████| 1/1 [00:55<00:00,  5.04s/it]\x1b[A\r100%|██████████| 1/1 [00:55<00:00, 55.58s/it]\n'

'863a392f-849b-4da7-b86e-4c0ce85eb221\n'

In [ ]:
# | exporti


@app.command("to-clickhouse")
@helper.requires_auth_token
def to_clickhouse(
    uuid: str = typer.Argument(
        ...,
        help="Prediction uuid.",
    ),
    host: str = typer.Option(..., help="Remote database host name."),
    database: str = typer.Option(..., help="Database name."),
    table: str = typer.Option(..., help="Table name."),
    protocol: str = typer.Option(..., help="Protocol to use (native/http)."),
    port: int = typer.Option(
        0,
        help="Host port number. If not passed, then the default value **0** will be used.",
    ),
    username: Optional[str] = typer.Option(
        None,
        "--username",
        "-u",
        help="Database username. If not passed, then the value set in the environment variable"
        " **CLICKHOUSE_USERNAME** will be used else the default value **root** will be used.",
    ),
    password: Optional[str] = typer.Option(
        None,
        "--password",
        "-p",
        help="Database password. If not passed, then the value set in the environment variable"
        ' **CLICKHOUSE_PASSWORD** will be used else the default value "" will be used.',
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output status only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Push the prediction results to a clickhouse database.

    If the database requires authentication, pass the username/password as commandline arguments or store it in
    the **CLICKHOUSE_USERNAME** and **CLICKHOUSE_PASSWORD** environment variables.
    """

    from airt.client import Prediction

    pred = Prediction(uuid=uuid)

    status = pred.to_clickhouse(
        host=host,
        database=database,
        table=table,
        port=port,
        protocol=protocol,
        username=username,
        password=password,
    )

    if quiet:
        status.wait()
        typer.echo(f"{pred.uuid}")
    else:
        typer.echo(
            f"Pushing the results for Prediction uuid: {pred.uuid} to the clickhouse database."
        )
        status.progress_bar()

In [ ]:
assert_has_help(["to-clickhouse"])

'Usage: root to-clickhouse [OPTIONS] UUID\n\n  Push the prediction results to a clickhouse database.\n\n  If the database requires authentication, pass the username/password as\n  commandline arguments or store it in the **CLICKHOUSE_USERNAME** and\n  **CLICKHOUSE_PASSWORD** environment variables.\n\nArguments:\n  UUID  Prediction uuid.  [required]\n\nOptions:\n  --host TEXT          Remote database host name.  [required]\n  --database TEXT      Database name.  [required]\n  --table TEXT         Table name.  [required]\n  --protocol TEXT      Protocol to use (native/http).  [required]\n  --port INTEGER       Host port number. If not passed, then the default value\n                       **0** will be used.  [default: 0]\n  -u, --username TEXT  Database username. If not passed, then the value set in\n                       the environment variable **CLICKHOUSE_USERNAME** will be\n                       used else the default value **root** will be used.\n  -p, --password TEXT  Database p

In [ ]:
# Tests for prediction to-clickhouse
# Testing positive scenario

with set_airt_service_token_envvar():
    with generate_prediction() as pred:
        # without -q
        cmd = [
            "to-clickhouse",
            pred.uuid,
            "--host",
            os.environ.get("CLICKHOUSE_HOST"),
            "--database",
            os.environ.get("CLICKHOUSE_DATABASE"),
            "--table",
            "test_clickhouse_push_prediction_airt_client",
            "--protocol",
            "native",
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0

        assert (
            f"Pushing the results for Prediction uuid: {pred.uuid} to the clickhouse database"
            in str(result.stdout)
        ), str(result.stdout)

        # with -q
        cmd = [
            "to-clickhouse",
            pred.uuid,
            "--host",
            os.environ.get("CLICKHOUSE_HOST"),
            "--database",
            os.environ.get("CLICKHOUSE_DATABASE"),
            "--table",
            "test_clickhouse_push_prediction_airt_client",
            "--protocol",
            "native",
            "-q",
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0

        assert f"{pred.uuid}" in str(result.stdout), str(result.stdout)

'Pushing the results for Prediction uuid: 863a392f-849b-4da7-b86e-4c0ce85eb221 to the clickhouse database.\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|██████████| 1/1 [00:05<00:00,  5.09s/it]\r100%|██████████| 1/1 [00:05<00:00,  5.09s/it]\n'

'863a392f-849b-4da7-b86e-4c0ce85eb221\n'